Chapitre 5 - Créer un outil CLI
===

# CLI ?

<img src="images/cli.png" align="right" width="200px" alt="Image de terminal" />

CLI (ou **C**ommand **L**ine **I**nterface) s'oppose à GUI (**G**raphical **U**ser **I**nterface) : c'est un outil dont l'utilisation se fera dans un terminal. Ce genre d'interface, quoiqu'abrupte pour les débutant-e-s est bien souvent nécessaire quand l'on travaille sur des serveurs distants : ces serveurs possèdent rarement une interface graphique et seul le terminal est accessible.

Par ailleurs, le développement d'interface graphique peut être très lourd dans les langages natifs, à tel point que bien souvent, aujourd'hui, on a recours à des outils comme [Electron](https://electronjs.org/) qui est en fait une installation simpliste de Chromium et a donc recours à JavaScript et HTML pour créer leur design.

Alors, dans la vie de tous les jours, pourquoi avoir un script accessible en ligne de commande ? Et bien tout simplement pour accélérer quelques tâches ingrates du quotidien, ou pour éviter à avoir à écrire un fichier python avec des paramètres changeant. Car, qui dit interface, dit bien souvent paramétrisation : l'interface développée permettra de personnaliser certains paramètres (où enregistrer le résultat d'un téléchargement par exemple).


# Développer un CLI

## Avant-propos

Dans ce cours, nous allons voir l'utilisation de [`Click`](http://click.palletsprojects.com/), une librairie à installer (dans votre environnement, faites `pip install click`). Qui dit librairie à installer dit généralement compétition et choix. Il existe donc d'autres librairies et je vous renvoie à cet article [*Building Beautiful Command Line Interfaces with Python*](https://codeburst.io/building-beautiful-command-line-interfaces-with-python-26c7e1bb54df) de Oyetoke Tobi Emmanuel dont une archive PDF est disponible dans le repository.

Le plus important de ces concurrents est *argparser* qui a l'avantage d'être, comme `json` et `csv`, une librairie de base de python et qui ne nécessite donc aucune installation. Une traduction du tutoriel officiel de python est disponible : https://docs.python.org/fr/3.5/howto/argparse.html

Le choix de Click s'est fait pour deux raisons : c'est la librairie utilisée par Flask, la librairie de création de sites web que nous utiliserons plus tard; elle gère les groupes de commande et possède une syntaxe très agréable.

## De Jupyter à l'IDE

À partir de maintenant, nous utiliserons un IDE (**I**nterface de **D**éveloppement **I**ntégrée). En tant qu'étudiant-e-s, vous avez accès à [PyCharm](https://www.jetbrains.com/pycharm/?fromMenu) pour lequel il existe une version *Community* (gratuite, limitée, suffisante) et une version Pro (accès via un compte éducation sinon paiement de licence).

Nous lancerons nos outils depuis le terminal et nous construirons peu à peu un outil permettant de faire une recherche rapide dans un fichier CSV représentant un texte lemmatisé.

## Commençons !

Pour créer un script, nous allons déjà créer le fichier : nous l'appelerons "recherche.py"

## Paramètre

## Option

## Typage de fichier

## Ajouter de la couleur !

## Raccourci Bash